# Document Similarity (English)
In this notebook, you will use the DocumentSimilarity tool to identify similar documents in the English language and decide whether to keep or remove them from the corpus.  

**Note:** this tool uses [MinHash](https://ekzhu.com/datasketch/minhash.html) to estimate the Jaccard similarity between sets of documents. MinHash is introduced by Andrei Z. Broder in this [paper](https://cs.brown.edu/courses/cs253/papers/nearduplicate.pdf).
<div class="alert alert-block alert-warning">
    
For any questions, feedback, and/or comments about the tool, please contact the Sydney Informatics Hub at [sih.info@sydney.edu.au](mailto:sih.info@sydney.edu.au?subject=[ATAP]%20Document%20Similarity%20Tool%20inquiry).</div>

<div class="alert alert-block alert-warning">
<b>Jupyter Notebook User Guide</b> 

If you are new to Jupyter Notebook, feel free to take a quick look at [this user guide](documents/jupyter-notebook-guide.pdf) for basic information on how to use a notebook.
</div>

<div class="alert alert-block alert-info">
<b>Document Similarity User Guide</b>

For instructions on how to use the Document Similarity tool, please refer to the [Document Similarity User Guide](documents/docsim-help-pages.pdf).
</div>

## 1. Setup
Before you begin, you need to import the DocumentSimilarity package and the necessary libraries and initiate them to run in this notebook.

In [ ]:
# import the DocumentSimilarity tool
print('Loading DocumentSimilarity...')
from atap_corpus_loader import CorpusLoader
from document_similarity import DocumentSimilarity

# initialize the DocumentSimilarity
ds = DocumentSimilarity()
print('Finished loading.')

## 2. Load the data
<div class="alert alert-block alert-warning">

This notebook will allow you to upload text data in one or more text files, each file contains one document. Alternatively, you can also upload multiple texts as an excel or CSV spreadsheet, in which each row is considered as one document ([see an example here](https://github.com/Sydney-Informatics-Hub/HASS-29_Quotation_Tool/blob/main/documents/sample_texts.xlsx)). Multiple files can be zipped and uploaded as a single archive file.
</div>
<div class="alert alert-block alert-warning">
    
Execute the next cell to run the [*ATAP corpus loader*](https://github.com/Australian-Text-Analytics-Platform/atap-corpus-loader) *UI* so that you can upload your files and build your corpus following the instructions below. Supported file types are .txt, .csv, .xlsx or a zip archive of these file types.  
Once a corpus is successfully built, you can continue with the rest of the notebook to run the Document Similarity Tool with your corpus.
</div>

<div class="alert alert-block alert-info">
<b>Using the ATAP corpus loader to load your dataset/corpus</b> <br>
<td><img src='./img/corpus_loader.png' style='width: 1000px'/></td> <br>
In order to use the document similarity tool, the user needs to build their corpus with the ATAP corpus loader. As a brief overview, the steps to use the loader are as follows (Please check the picture above for locations referred by superscript numbers):

1. Upload your text files using the file explorer pane on the left<sup>1</sup>. <p>If the pane is not activated, clicking on the folder icon<sup>2</sup> will show you the file explorer pane.
Files can be uploaded into any folder by either drag-n-drop or via the upload button<sup>3</sup> <br>
Wait until your corpus is uploaded before you return to the notebook and execute the codes.

2. Execute the following code cell to run the ATAP Corpus Loader in order to build your corpus from the uploaded files, all supported filetypes will be displayed and can be filtered<sup>4</sup> in the corpus-loader.

3. Choose the files in the selector pane<sup>5</sup>, then click the 'Load as corpus' button<sup>6</sup>.  
If loading from a spreadsheet with multiple columns, first, select the correct header of the column that contains the text data<sup>7</sup>. Then make sure the required metadata columns are checked<sup>8</sup> with the correct datatype<sup>9</sup> for your corpus.  
For example, if one column consists of text, the datatype TEXT is appropriate and no changes are necessary.  
If plain text files are loaded, the Corpus Loader also automatically creates and includes the filename as TEXT type metadata.

4. Give your corpus a name<sup>10</sup> and click on the button “Build corpus”<sup>11</sup>. Wait until you receive the message “Corpus … built successfully”.   
Review your corpus in the Corpus Overview or continue immediately to the next code cell in the notebook.  
Refer to the screenshot above for each necessary operation.<br>

</div>

<div class="alert alert-block alert-warning">

More information about the ATAP corpus loader is provided in the [User Guide](documents/Corpus%20Loader%20User%20Guide.pdf).
</div>


In [ ]:
corpus_loader: CorpusLoader = CorpusLoader("./", )
corpus_loader

<div class="alert alert-block alert-warning">
    
**Automatic deduplication of identical documents within the corpus**
    
The Document Similarity Tool is designed to find the documents in your corpus that are similar, but not 100% identical. As a first step, the tool will therefore aim to identify all identical documents in the corpus and undertake an automatic deduplication. For these identical documents, only the first document (according to alphabetical order by “text_name” or filename) will be retained in the corpus. The Jaccard-based similarity analysis below is then only run on the deduplicated version of the corpus to avoid including identical documents in the pairwise display. You can see the names of all identical documents in your corpus by executing the following cell, which allows you to export the relevant table as a CSV file. This table provides the filename of the retained file in the ‘kept’ column and the file names of the relevant identical (excluded) files in subsequent numbered columns. For example, the column ‘1’ contains the file name of the first duplicate of the file in the ‘kept’ column, and so on – this depends on the number of duplicates identified.

</div>

In [ ]:
# Grab the corpus that was last built for processing. 
# Alternatively, one can replace the first line of code with "corpus = corpus_loader.get_corpus('corpusname')"
corpus = corpus_loader.get_latest_corpus()
ds.set_text_df(corpus)

In [ ]:
# display uploaded text
n=5

ds.text_df.head()

## 3. Calculate Document Similarity
Once your texts have been uploaded, you can begin to calculate the similarity between documents in the corpus. 

<div class="alert alert-block alert-info">
<b>Tools:</b>    

- MinHash: fast implementation of estimating Jaccard similarity between documents in the corpus.  
- Gensim: to tokenize the text.  
    
<b>Note:</b> in general, Gensim splits the text whenever whitespace or punctuation is encountered and digits are excluded, e.g., the text "Here's to victory no 2" will be tokenized into five tokens: "Here", "s", "to", "victory" and "no". For more information, please visit [this page](https://radimrehurek.com/gensim/utils.html#gensim.utils.tokenize).
</div>

<div class="alert alert-block alert-danger">
<b>Memory limitation in Binder</b> 
    
The free Binder deployment is only guaranteed a maximum of 2GB memory. Processing very large text files may cause the session (kernel) to re-start due to insufficient memory. Check [the user guide](https://github.com/Australian-Text-Analytics-Platform/semantic-tagger/blob/main/documents/jupyter-notebook-guide.pdf) for more info. 
</div>

<div class="alert alert-block alert-warning">
<b>Parameters for calculating similarity</b> 
    
The DocumentSimilarity tool uses Jaccard similarity to measure the similarity between documents. In the code below, we have specified and explained the default parameters for calculating the Jaccard similarity. However, you can also change these parameters should you wish. 
</div>

In [ ]:
# USER SPECIFIES THESE VARIABLES
# set the n-gram size (the number of words used to detect similarity), 
# e.g., n-gram=1 means compare every word ('apple' and 'orange'), 
# n-gram=2 means compare every pair of words ('one apple' and 'two oranges'), etc.
ngram_value = 1

# select whether to calculate actual or estimated Jaccard similarity 
# to measure the similarity between documents 
# we recommend using estimated Jaccard similarity for large corpus of documents (faster)
actual_jaccard = False # True or False

# whether to exclude punctuations when calculating Jaccard similarity
ds.exclude_punc = True # True or False

# set the number of permutation functions (num_perm) parameter for estimating Jaccard similarity
# higher permutation functions improves the accuracy, but also increases query cost
num_perm = 256

# anything with >= the cutoff will be identified as similar documents
similarity_cutoff = 0.6 # value should be between 0-1

In [ ]:
# begin the process of calculating similarity and identify similar documents
ds.calculate_similarity(ngram_value, num_perm, similarity_cutoff, actual_jaccard)

## 3. Analyse similar documents
Once the tool has finished calculating the document similarity, you can begin to analyse the outcome.  

The graph below is a histogram of the count of similar documents in the corpus as measured by their Jaccard similarity. In this histogram, you can identify how many documents are found at different level of similarity measures.

<div class="alert alert-block alert-warning">
<b>Histogram of similar documents</b> 
    
The x-axis on the histogram shows the Jaccard similarity scores for every document in the corpus, and the y-axis (the height of the bar) tells us how many similar documents are found at those Jaccard similarity score ranges. 
</div>

In [ ]:
# plot the similarity count accross the entire corpus
ds.plot_hash_similarity_by_source(ds.deduplication_df)

<div class="alert alert-block alert-warning">
<b>Heatmap of similar documents</b> 
    
The below heatmap shows the Jaccard similarity scores between pair of similar documents, with the x- and y-axes showing the text_id of the similar document pairs (you can hover over the similar nodes to display the text name pairs). Please note that the heatmap only displays pair of similar documents with scores above the similarity cut-off, as defined earlier.  
</div>  

<div class="alert alert-block alert-danger">
<b>Large number of similar documents</b> 
    
You can resize the heatmap, adjust the font size or the font color to better visualize your data by specifying the below parameters. You can also zoom in/out of the heatmap, move it around, save and download it to your local computer using the interactive tool on the right hand-side of the heatmap.  

<b>Note:</b> visualizing a large number of similar document pairs (**>500**) may slow down the notebook.   
</div>
<div class="alert alert-block alert-info">
<b>Input before plotting</b> 
    
To avoid plotting oversized figure, the user is asked to **specify the range** of matched documents to be included in the heatmap.
Entering **'n'** will cancel the figure generation.
Entering **'y'** will proceed with **all pairs** of similar documents.
Entering an **integer number**, such 30, will generate the figure with the top-30 pairs of the similar documents.
Entering a number range like **15-45** will generate the figure with the selected range (15 to 45) of the document pairs.
    
**Press Enter Key after inputting.**
</div>

In [ ]:
# define the plot width, height, font size and color
plot_width = 900 # increase plot width if necessary
plot_height = 800 # increase plot height if necessary
font_size = '14px'
text_color = 'white' # 'black' or 'white' would usually work for most scenarios

print('\033[1mVisualizing a large number of similar document pairs (>500) may slow down the notebook.\033[0m')
print('There are \033[1m{}\033[0m document pairs in the current process'.format(ds.deduplication_df.shape[0]))
plot_range = input("""Enter the range of documents pairs to be plotted, e.g. y, n, 10-25, or 30.""")

# plot heatmap of Jaccard similarity
ds.plot_heatmap_similarity(similarity_cutoff,
                                plot_width,
                                plot_height,
                                font_size,
                                text_color,
                                plot_range)

<div class="alert alert-block alert-warning">
<b>Analyse similar documents</b> 

Below you can generate a list of similar documents (in pairs) found by the tool, based on the similarity cutoff specified earlier. By default, the tool makes recommendations on whether to 'keep' or 'remove' each similar document (the tool will recommend to remove the document with the lower word count, if the Jaccard similarity is above the specified threshold). However, using the below tool, you can generate each pair of similar documents (by specifying the row index you wish to analyse), analyse them, and update the action/recommendation as you see fit.
</div>

<div class="alert alert-block alert-danger">
<b>Similar documents table</b> 

The table below displays only those texts identified as similar based on the Jaccard similarity cut-off selected earlier and the number of texts included in the table display therefore also informs you how many texts in your corpus are identified as within the cut-off threshold.
</div>

In [ ]:
ds.display_deduplication_text()

<div class="alert alert-block alert-warning">
<b>What information is included in the above table?</b> 

**text_id1/2:** the text id of the pair of similar documents.
    
**text_name1/2:** the text name of the pair of similar documents.
   
**word_count1/2:** the word count of the pair of similar documents.

**status1/2:** whether to 'keep' or 'remove' each similar document.

**similarity:** the Jaccard similarity between the pair of similar documents.
</div>

<div class="alert alert-block alert-danger">
    
**Caveat: Discrepancies in the highlighted side-by-side comparison**

In the display of document pairs where differences between texts are highlighted for checking by users, only document pairs based on the Jaccard similarity parameters are included. However, this visualisation uses the python function difflib which is independent from the Jaccard calculation and may thus highlight differences in punctuation (regardless of previous settings) and this function may also at times contain incorrectly highlighted text blocks. Despite this caveat, the visualisation should still be helpful in allowing you to decide which of the two texts you want to ‘keep’ or ‘remove’.
 
</div>

## 5. Save duplicated/non-duplicated texts
Once you are happy with the list of texts that you want to keep, you can run the below code to save the non-duplicated texts (those with 'keep' status) or the duplicated ones (those with 'remove' status) into a zip of text (.txt) files and download them to your local computer.

Remember that identical texts were already removed in the first step above and that this process of checking for similar texts was only run on the remaining (non-identical) texts. You can see the number of saved files in the progress bar which appears after you click ‘Save non-duplicated texts’ or after you click ‘Save duplicated texts’.

In [ ]:
rows_to_display=5

ds.finalise_and_save(rows_to_display)